## Functions
- Defined using **DEFUN** macro
- Basic skeleton:
```lisp
(defun name (parameter*)
    "Optional documentation string"
    body-form*)
```
- The body-form gets evaluated in order and the value of the last expressionn is returned as the function value. Else, the **RETURN-FROM** special operator can be used to return from anywhere in the program.

### Function Parameter Lists
#### Required Parameters
- When the parameter list is a simple list of variable names, the parameters are **_required parameters_**. Lisp throws an error if required parameters are missing.

#### Optional Parameters
- To define a function with optional parameters, after the names of any required parameters, place the symbol **&optional** followed by the names of the optional parameters, e.g.:
```lisp
(defun foo (a b &optional c d) (list a b c d))
```
If there are no arguments to bind to the optional parameters, they default to **NIL**.
```lisp
(foo 1 2)     ; ==> (1 2 NIL NIL)
(foo 1 2 3)   ; ==> (1 2 3 NIL)
(foo 1 2 3 4) ; ==> (1 2 3 4)
```

- Lisp will throw an error if called with too few or too many parameters, e.g. if `foo` is called with more than 4 or less than 2 arguments.

- You can define a different default value than **NIL**. To do so, replace the optional parameter name with a list containing the name and an expression. The expression will be evaluated only if the caller does not pass enough arguments to provide a value for the optional parameter.
```lisp
(defun foo (a &optional (b 10)) (list a b))
(foo 1 2) ; ==> (1 2)
(foo 1)   ; ==> (1 10)
```
You can also compute a default value based on the value of other parameters, e.g.
```lisp
(defun make-rectangle (width &optional (height width)) ...)
```
Unless explicitly passed, the height will take the value of the width.

- If you need to know if the value of an optional parameter was supplied by the caller or is the default value, you can add another variable name to the parameter specifier after the default value expression. This other variable will be bound to true if the caller actually supplied an argument for this parameter and **NIL** otherwise. By convention, that variable is named the same as the actual parameter with a "_-supplied-p_" at the end, e.g.
```lisp
(defun foo (a b &optional (c 3 c-supplied-p))
    (list a b c c-supplied-p))
```
```lisp
(foo 1 2)   ; ==> (1 2 3 NIL)
(foo 1 2 3) ; ==> (1 2 3 T)
(foo 1 2 4) ; ==> (1 2 4 T)
```

#### Rest parameters
- A function can handle a variable number of parameters. To do so, you include a catchall parameter after the symbol **&rest** in the parameter list. For example, the **+** function can take a variable number of arguments:
```lisp
(+)
(+ 1)
(+ 2 3)
(+ 45 2 3 4 5)
```
It's signature probably looks like:
```lisp
(defun + (&rest numbers) ...)
```

#### Keyword parameters
- Allow the caller to specify which values go with which parameters.
- To give a function keyword parameters, after any required, &optional and &rest parameters, include the symbol **&key** and then any number of keyword parameter specifiers, e.g.
```lisp
(defun foo (&key a b c)
    (list a b c))
```
When this function is called, each keyword parameter is bound to the value immediately following a keyword of the same name. **Keywords** are names that start with a colon and are automatically defined as self-evaluating constants.
```lisp
(foo)                ; ==> (NIL NIL NIL)
(foo :a 1)           ; ==> (1 NIL NIL)
(foo :b 1)           ; ==> (NIL 1 NIL)
(foo :c 1)           ; ==> (NIL NIL 1)
(foo :a 1 :c 3)      ; ==> (1 NIL 3)
(foo :a 1 :b 2 :c 3) ; ==> (1 2 3)
(foo :a 1 :c 3 :b 2) ; ==> (1 2 3)
```

- Keyword parameters can also provide default values and the name of a "supplied-p" variable. In both keyword and optional parameters, the default value form can refer to parameters appearing earlier in the parameter list, e.g
```lisp
(defun foo (&key ((:apple a)) ((:box b)) ((:charlie c) 0 c-supplied-p))
    (list a b c c-supplied-p))
```
This can be called as:
```lisp
(foo :apple 10 :box 12 :charlie 20)  ; ==> (10 12 20)
```
Useful when you want to decouple public API of the function from the internal details. _Not very frequently used._

### Function Return Values
- By default, all functions will use the value of the last evaluated expression as their return value.
- Can also use the **RETURN-FROM** special operator to return from within a function.
- **RETURN-FROM** is a special operator whose first argument is the name of the block from which to return. e.g.
```lisp
(defun foo (n)
    (dotimes (i 10)
        (dotimes (j 10)
            (when (> (* i j) n)
                (return-from foo (list i j))))))
```
`foo` finds the first pair of numbers less than 10 whose product is greater than the argument.

### Functions As Data (Higher Order Functions)
- You can use the special operator **FUNCTION** to get at a function object. The equivalent syntactic sugar is using **#'**.
```
CL-USER> (defun bar (x) (* x 2))
BAR
CL-USER> (function bar)
#<FUNCTION BAR>
CL-USER> #'bar
#<FUNCTION BAR>
CL-USER>
```

- You can invoke a function object using either of the functions **FUNCALL** or **APPLY**.
- Use **FUNCALL** if you know the number of arguments you're going to pass to the function when you're writing the code. First argument to **FUNCALL** is the function object to invoke. The rest of the arguments are pass to the invoked function. The following are equivalent function calls:
```lisp
(foo 1 2 3)
(funcall #'foo 1 2 3)
```
- A more apt use of FUNCALL is to invoke functions passed as parameters:
```
CL-USER> (defun plot (fn min max step)
             (loop for i from min to max by step do
                (loop repeat (funcall fn i) do (format t "+"))
                (format t "~%")))
PLOT
CL-USER> (plot #'exp 0 4 1/2)
+
++
+++
+++++
++++++++
+++++++++++++
+++++++++++++++++++++
++++++++++++++++++++++++++++++++++
+++++++++++++++++++++++++++++++++++++++++++++++++++++++
NIL
CL-USER>
```

- If the arguments are in a list, use **APPLY** instead of **FUNCALL**.
```lisp
(apply #'plot plot-data)
```

### Anonymous Functions
- You can define anonymous functions using `lambda`. General form is:
```lisp
(lambda (parameters) body)
```

```lisp
(funcall #'(lambda (x y) (+ x y)) 2 3)  ;  ==> 5
```